In [1]:
# Importació de llibreries per al processament de text,
# reducció de dimensionalitat, classificació i validació

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# Càrrega del conjunt de dades pre-processat i reduït
# Es tracta d'un subconjunt del corpus Sentiment140 amb textos i etiquetes

df_reduït = pd.read_csv('twitter_reduït_Dataset.csv', encoding='latin-1')

# Separació de les variables predictives (textos) i de la variable objectiu (sentiment)
X = df_reduït['text'].values
y = df_reduït['target'].values

# Definició d’una llista de paraules buides personalitzades
# Aquestes paraules es filtraran durant la vectorització per no aportar informació rellevant

stopwords_personalitzades = [
    'as', 'an', 'the', 'in', 'on', 'at', 'to', 'of', 'and', 'or',
    'is', 'it', 'for', 'with', 'that', 'this', 'was', 'be',
    'are', 'were', 'been', 'from', 'by', 'about', 'into', 'out',
    'up', 'down', 'over', 'under', 'then', 'than', 'so', 'but', 'not'
]


In [2]:
i=0
# Definició de paràmetres a explorar:
# k: nombre de temes a extreure mitjançant LDA
# C: paràmetre de regularització per a la regressió logística

valors_k = [10]
valors_C = [1]

# Inicialització de la llista on es guardaran els resultats
resultats = []

# Configuració de la validació creuada en 5 particions

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Bucle principal d'experimentació per a cada combinació (k, C)

for k in valors_k:
    for C in valors_C:
        puntuacions = []  # Llista per desar les puntuacions de cada fold

        # Bucle de validació creuada
        for train_idx, test_idx in cv.split(X, y):
            # Extracció dels textos i etiquetes per entrenament i test
            X_train_raw, X_test_raw = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            # Vectorització del text amb CountVectorizer
            # S’ajusta únicament amb el conjunt d’entrenament per evitar data leakage

            vectoritzador = CountVectorizer(max_features=10000, stop_words=stopwords_personalitzades)
            X_train_counts = vectoritzador.fit_transform(X_train_raw)
            X_test_counts = vectoritzador.transform(X_test_raw)

            # Reducció de dimensionalitat amb LDA (Latent Dirichlet Allocation)
            # S’extreuen k temes latents a partir del conjunt d'entrenament

            lda = LatentDirichletAllocation(n_components=k, random_state=42)
            X_train_topics = lda.fit_transform(X_train_counts)
            X_test_topics = lda.transform(X_test_counts)

            # Entrenament del model de regressió logística
            # S’utilitza regularització L2

            model = LogisticRegression(
                penalty='l2',
                C=C,
                solver='lbfgs',
                max_iter=100,
                random_state=42,
            )
            model.fit(X_train_topics, y_train)
            print("Iteracio: "+str(i) )
            i=i+1

            # Predicció de probabilitats i càlcul de la mètrica AUC-ROC
            # Aquesta mètrica avalua la capacitat de separació del model

            probabilitats = model.predict_proba(X_test_topics)[:, 1]
            puntuacio = roc_auc_score(y_test, probabilitats)
            puntuacions.append(puntuacio)
            print("AUC-ROC: Fold " + str(i) +  ":" + str(puntuacio))  # Impressió de la puntuació AUC del fold actual

        # Emmagatzematge del resultat mitjà per a la combinació de paràmetres actual

        resultats.append({
            'k': k,
            'C': C,
            'auc_roc_mitjana': np.mean(puntuacions)
        })

        # Impressió dels resultats parcials per monitoritzar el progrés
        print(resultats)


Iteracio: 0
AUC-ROC: Fold 1:0.58601328125
Iteracio: 1
AUC-ROC: Fold 2:0.6121251953125
Iteracio: 2
AUC-ROC: Fold 3:0.6098244140625
Iteracio: 3
AUC-ROC: Fold 4:0.5948289062500001
Iteracio: 4
AUC-ROC: Fold 5:0.61718828125
[{'k': 10, 'C': 1, 'auc_roc_mitjana': np.float64(0.603996015625)}]


In [3]:
# Organitzem els resultats
df_resultats = pd.DataFrame(resultats)

# Mostrar la taula de resultats
print("\n Resultats de validació creuada (mitjana AUC-ROC per a k=10):\n")
print(df_resultats.to_string(index=False, float_format="%.4f"))


 Resultats de validació creuada (mitjana AUC-ROC per a k=10):

 k  C  auc_roc_mitjana
10  1           0.6040
